In [2]:
import numpy as np

In [3]:
n = 100

def foo(i, j, k):
    return i + j + k

A = np.fromfunction(foo, [n, n, n])
d = len(A.shape)

# Utils

In [85]:
def crop_vector(a: np.array, eps: float) -> int:
    """crop vector of positive values until all sum minus prefix sum < eps

    Args:
        a (np.array): vector of positive values
        eps (float): tolerance

    Returns:
        int: len of prefix sum
    """    
    r = 0
    cur_sum = 0
    sum = np.sum(a)

    while r < len(a):
        cur_sum += a[r]
        if sum - cur_sum < eps:
            return r + 1
        r += 1
    return r

def tensor_unfold(A: np.array, k : int) -> np.array:
    """unfold tensor by dimension k

    Args:
        A (np.aray): of size (n1, n2, ..., nd)
        k (int): dimension for unfold

    Returns:
        aray: matrix (nk, -1)
    """    
    return np.reshape(np.moveaxis(A, k, 0), (A.shape[k], -1))

def mul_by_dim(T: np.array, A: np.array, k: int)-> np.array:
    """multiply tensor by matrix on dimenson k

    Args:
        T (np.array): Tensor with shape (n1, ... nd)
        A (np.array): Matrix with shape (nk, mk)
        k (int): dimension

    Returns:
        np.array: Tensor with shape (n1, ... , mk, ... nd)
    """    
    d = len(T.shape)
    return np.einsum(T, [i for i in range(d)], A, [k, d], [i if i != k else d for i in range(d)])

# st HO SVD

In [90]:
def stHOSVD(A, eps=1e-2):
    d = len(A.shape)
    U = []
    G = A
    shape = [k for k in A.shape]
    for k in range(d):
        Ak = tensor_unfold(G, k)
        u, s, v = np.linalg.svd(Ak, full_matrices=False)
        r = crop_vector(s, eps / d**0.5)
        shape[k] = r
        U.append(u[:, :r].T)
        G = np.diag(s[:r]) @ v[:r, :]
        G = G.reshape(r, *[shape[i] for i in range(len(shape)) if i != k])
    return np.transpose(G), U  # ranks are inverse in G  


In [91]:
g, U = stHOSVD(A)

In [88]:
T = g
for k in range(d):
    T = mul_by_dim(T, U[k], k)

np.linalg.norm(T - A)

7.954796900416866e-10

# Recompress

In [92]:
def recompress(G, U, eps):
    Gnew = G.copy()
    d = len(G.shape)
    V = []
    Q = []
    #reortoganization
    for k in range(d):
        q, r = np.linalg.qr(U[k].T) # UT = QR
        Q.append(q)
        Gnew = mul_by_dim(Gnew, r.T, k) # U = RT QT
    Gnew, W = stHOSVD(Gnew, eps)
    for k in range(d):
        V.append(W[k] @ Q[k].T)
    return Gnew, V

In [93]:
r = 5
N = 100

g = np.random.randn(r, r, r)
U = [np.random.randn(r, N) for _ in range(3)]

In [94]:
g1, U1 = recompress(g, U, 1e-5)

In [96]:
T0 = g
for k in range(d):
    T0 = mul_by_dim(T0, U[k], k)

T1 = g1
for k in range(d):
    T1 = mul_by_dim(T1, U1[k], k)


np.linalg.norm(T1 - T0)

3.147727427093443e-11